In [1]:
exec(open("initKeras.py").read())

Using TensorFlow backend.


In [2]:
# model numbers to retrain, set to false or blank if you want to retrain all the models in DoodleModels. Strictly a list of integers
trainOnlyModels = [1,2,3,4,5,6,7,8]
saveModels = True

# #data preparation
inputDir = "C:\\tim209\\filtered by category\\output\\*.npy"
modelDirectory = "F:\\myProjects\\tim209\\doodle\\savedModels\\"
modelGraphDirectory = "F:\\myProjects\\tim209\\doodle\\experiment results\\"
dataUtils = dataUtils( inputDir )
dataUtils.process()

Processing aircraft carrier C:\tim209\filtered by category\output\aircraft carrier.npy
Observation #: 50000
label hot vectors #: 50000
Processing airplane C:\tim209\filtered by category\output\airplane.npy
Observation #: 50000
label hot vectors #: 50000
Processing alarm clock C:\tim209\filtered by category\output\alarm clock.npy
Observation #: 50000
label hot vectors #: 50000
Processing ambulance C:\tim209\filtered by category\output\ambulance.npy
Observation #: 50000
label hot vectors #: 50000
Processing angel C:\tim209\filtered by category\output\angel.npy
Observation #: 50000
label hot vectors #: 50000
Processing animal migration C:\tim209\filtered by category\output\animal migration.npy
Observation #: 50000
label hot vectors #: 50000
Processing ant C:\tim209\filtered by category\output\ant.npy
Observation #: 50000
label hot vectors #: 50000
Processing anvil C:\tim209\filtered by category\output\anvil.npy
Observation #: 50000
label hot vectors #: 50000
Processing apple C:\tim209\fil

In [3]:
(images, labels, classToLevels ) = dataUtils.getData()
(imagesR, labelsR, classToLevelsR ) = dataUtils.getRandomizedData()
images = dataUtils.reshapeDataForKeras( images )
imagesR = dataUtils.reshapeDataForKeras( imagesR )

(1200000, 784) (1200000, 24) {'aircraft carrier': 0, 'airplane': 1, 'alarm clock': 2, 'ambulance': 3, 'angel': 4, 'animal migration': 5, 'ant': 6, 'anvil': 7, 'apple': 8, 'arm': 9, 'asparagus': 10, 'axe': 11, 'backpack': 12, 'bed': 13, 'bench': 14, 'binoculars': 15, 'bird': 16, 'butterfly': 17, 'rollerskates': 18, 'school bus': 19, 'square': 20, 'steak': 21, 'stove': 22, 'strawberry': 23}


In [4]:
doodleModels = DoodleModels(images, labels)

train X: (960000, 28, 28, 1)
test X: (240000, 28, 28, 1)
train Y: (960000, 24)
test Y: (240000, 24)


In [ ]:
epochSize = 15
batchSize = 64
learningRate = 0.001
confStr = "_e" + str( epochSize ) + "_b" + str( batchSize ) + "_lr" + str( learningRate ) + " "
histories = []
for i in range( doodleModels.numberOfAvailableModels ):
    
    modelNo = i + 1
    
    #1 filter if needed
    if type( trainOnlyModels ) == list and len( trainOnlyModels ) > 0 and modelNo not in trainOnlyModels:
        print( "Skipping model #", modelNo )
        continue
        
    #2 get and train model
    model = doodleModels.getModel( modelNo, learningRate )
    history, testLoss, testAcc = doodleModels.runModel( model, epochSize, batchSize )
    histories.append( ( model, history, testLoss, testAcc ) )
    
    #3 save model performance plot
    plotSavePath = modelGraphDirectory + model.name + confStr + ".png"
    doodleModels.plotModelTrainPerformance( model, plotSavePath )
    
    #4 save trained model
    if saveModels == True:
        modelPath = modelDirectory + model.name + confStr + "_Pa.h5"
        doodleModels.saveModelAndHistory( model, modelPath )


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 32)          9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                9248      
__________

In [ ]:
#print all model graph
plt.close()
colorPalette = sns.color_palette("hls", len( histories ) )
colorIndex = 0
for ( model, history, testLoss, testAcc ) in histories:
    epochX = np.arange( len( history.epoch ) ) + 1
    fig = plt.figure( figsize=( 16, 10 ) )
    plt.figure(1)
    
    if 'val_loss' in history.history:
        plt.plot( epochX, history.history['val_categorical_accuracy'], color = colorPalette[ colorIndex] , label = model.name + " - acc" )
        plt.plot( epochX, history.history['val_loss'], color = colorPalette[ colorIndex], label = model.name + " - loss", linestyle = '--' )
    else:
        plt.plot( epochX, history.history['categorical_accuracy'], color = colorPalette[ colorIndex] , label = model.name + " - acc" )
        plt.plot( epochX, history.history['loss'], color = colorPalette[ colorIndex], label = model.name + " - loss", linestyle = '--' )

    plt.title( 'Epoch vs Test: All Models -' + confStr )
    plt.xlabel( "Epoch" )
    plt.ylabel( "amount" )
    plt.xticks( epochX )
    plt.xlim( right = len( history.epoch ) * 1.2 )
    plt.legend()
    colorIndex = colorIndex + 1

allPath =  modelGraphDirectory + "all_" + confStr + datetime.now().strftime( "%Y-%M-%d %Hh%Mm%Ss" ) + ".png"
plt.savefig( allPath )